# ДЗ №-5 по MLOps

## Цель задания

Применить средства автоматизации тестирования python для автоматического тестирования качества работы модели машинного обучения на различных датасетах.

## Содержание задания

*   Создать три датасета с «качественными данными», на которых можно обучить простую модель линейной регрессии.
*   На одном из этих датасетов обучить модель линейной регрессии.
*   Создать датасет с шумом в данных.
*   Провести тестирование работы модели на разных датасетах с использованием Pytest, анализируя качество предсказания, обнаружить проблему на датасете с шумами.

## Import библиотек

In [1]:
!pip install ipytest

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.0 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import ipytest

### 1. Создание исходного датасета

In [3]:
# Установка параметров
np.random.seed(42)  # для воспроизводимости результатов
num_samples = 100  # количество примеров

# Генерация случайных признаков
data = {
    'Feature_1': np.random.rand(num_samples),
    'Feature_2': np.random.rand(num_samples),
    'Feature_3': np.random.rand(num_samples),
    'Feature_4': np.random.rand(num_samples),
    'Feature_5': np.random.rand(num_samples),
    'Feature_6': np.random.rand(num_samples),
    'Target': np.random.rand(num_samples) * 10  # Целевая переменная, например, цены
}

# Создание DataFrame
df = pd.DataFrame(data)

# Показать первые несколько строк DataFrame
print(df.head())

   Feature_1  Feature_2  Feature_3  Feature_4  Feature_5  Feature_6    Target
0   0.374540   0.031429   0.642032   0.051682   0.103124   0.698162  1.689351
1   0.950714   0.636410   0.084140   0.531355   0.902553   0.536096  2.785903
2   0.731994   0.314356   0.161629   0.540635   0.505252   0.309528  1.770105
3   0.598658   0.508571   0.898554   0.637430   0.826457   0.813795  0.887025
4   0.156019   0.907566   0.606429   0.726091   0.320050   0.684731  1.206359


### 2. Обучение модели линейной регрессии

In [4]:
# Разделение данных на обучающую и тестовую выборку
X = df.drop('Target', axis=1)
y = df['Target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели линейной регрессии
model = LinearRegression()
model.fit(X_train, y_train)

# Оценка модели на тестовой выборке
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print("MSE on Test Data:", mse)

MSE on Test Data: 7.822410653876713


### 3. Создание датасета с шумом

In [5]:
# Генерация шума для признаков
noise_level = 0.1  # Уровень шума
feature_noise = np.random.randn(num_samples, 6) * noise_level

# Добавление шума к данным
noisy_features_data = data.copy()
for i in range(1, 7):
    noisy_features_data[f'Feature_{i}'] += feature_noise[:, i-1]

# Создание DataFrame с шумными признаками
df_noisy_features = pd.DataFrame(noisy_features_data)
print("DataFrame with noisy features:")
print(df_noisy_features.head())


DataFrame with noisy features:
   Feature_1  Feature_2  Feature_3  Feature_4  Feature_5  Feature_6    Target
0   0.309357   0.036169   0.555990   0.013226   0.203753   0.640473  1.689351
1   1.034284   0.523440   0.137120   0.675511   0.655388   0.456407  2.785903
2   0.789701   0.294051   0.198743   0.480237   0.513911   0.293960  1.770105
3   0.715437   0.534013   0.932314   0.596242   0.777697   0.770539  0.887025
4   0.195464   0.865468   0.635407   0.933631   0.407162   0.652129  1.206359


In [6]:
# Генерация шума для целевой переменной
target_noise = np.random.randn(num_samples) * noise_level * 10  # Больший шум для целевой переменной

# Добавление шума к целевой переменной
noisy_target_data = data.copy()
noisy_target_data['Target'] += target_noise

# Создание DataFrame с шумной целевой переменной
df_noisy_target = pd.DataFrame(noisy_target_data)
print("DataFrame with noisy target:")
print(df_noisy_target.head())


DataFrame with noisy target:
   Feature_1  Feature_2  Feature_3  Feature_4  Feature_5  Feature_6    Target
0   0.309357   0.036169   0.555990   0.013226   0.203753   0.640473  1.445194
1   1.034284   0.523440   0.137120   0.675511   0.655388   0.456407  3.749991
2   0.789701   0.294051   0.198743   0.480237   0.513911   0.293960  2.959575
3   0.715437   0.534013   0.932314   0.596242   0.777697   0.770539 -0.340582
4   0.195464   0.865468   0.635407   0.933631   0.407162   0.652129  1.803759


### 4. Написание тестов с использованием pytest

In [8]:
ipytest.autoconfig()

def test_model_on_noisy_features():
    X_noisy_features = df_noisy_features.drop('Target', axis=1)
    y_noisy_features = df_noisy_features['Target']
    _, X_test_noisy_features, _, y_test_noisy_features = train_test_split(X_noisy_features, y_noisy_features, test_size=0.2, random_state=42)
    predictions = model.predict(X_test_noisy_features)
    mse = mean_squared_error(y_test_noisy_features, predictions)
    print("MSE on Noisy Features Data:", mse)
    assert mse > 5 # тест проходит

def test_model_on_noisy_target():
    X_noisy_target = df_noisy_target.drop('Target', axis=1)
    y_noisy_target = df_noisy_target['Target']
    _, X_test_noisy_target, _, y_test_noisy_target = train_test_split(X_noisy_target, y_noisy_target, test_size=0.2, random_state=42)
    predictions = model.predict(X_test_noisy_target)
    mse = mean_squared_error(y_test_noisy_target, predictions)
    print("MSE on Noisy Target Data:", mse)
    assert mse < 5 # тест провален

ipytest.run('-v')


======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.5.0
rootdir: /content
plugins: anyio-3.7.1
collected 2 items

t_81b67dc5716c4d7c80e31360fb996e30.py .F                                                     [100%]

============================================= FAILURES =============================================
____________________________________ test_model_on_noisy_target ____________________________________

    def test_model_on_noisy_target():
        X_noisy_target = df_noisy_target.drop('Target', axis=1)
        y_noisy_target = df_noisy_target['Target']
        _, X_test_noisy_target, _, y_test_noisy_target = train_test_split(X_noisy_target, y_noisy_target, test_size=0.2, random_state=42)
        predictions = model.predict(X_test_noisy_target)
        mse = mean_squared_error(y_test_noisy_target, predictions)
        print("MSE on Noisy Target Data:", mse)
>       asser

<ExitCode.TESTS_FAILED: 1>